# TampereBNB Listings - Price Prediction
<p> Get ready for an exhilarating data science adventure! In this exciting assignment, you will dive into the world of TampereBNB, the popular platform for short-term accommodation rentals. 
    <br>
    Your mission? To analyze data from this platform and use your data science skills to predict missing prices for some of the listings, using the tools mentioned in the following cell. </p>
<br>

## Instructions
- Train a regression model of your choice on predicting the listing prices of the training data. 
- Use the trained model to get the price predictions for the listings in the testing data.
- Store the resulting dataframe as a pickled (out.pkl) file. 

**NOTE: The code snippets for loading the data files and outputting the resulting dataframe, are provided. Do not update them.**


#### Accessing the dataset
To facilitate your work, we have created two separate training and testing TampereBNB csv files, located within the `data/` folder. Make sure the path to the files is the same, before submitting your solution.

#### TODO

- You are expected to predict prices for the listings on the testing data by using the following libraries (Besides the built-in python modules, specific libraries can be included upon request):
    - scikit-learn (sklearn)
    - pandas
    - numpy
    
- Store your predictions as a dataframe with the attribute `Hinta` (case sensitive).
- Save the dataframe in a pickle file, `out.pkl` (case sensitive).



## Importing the required packages

In [11]:
import sklearn
import numpy as np
import pandas as pd 

## Loading Data

In [12]:
# !MAKE SURE TO NOT CHANGE THE CODE WITHIN THIS CELL!. 
# Instead, put the data files within a folder named 'data' such that the paths would work.
training_df = pd.read_csv('./data/Tampere_BNB_training_listing.csv')
testing_df = pd.read_csv('./data/Tampere_BNB_testing_listing.csv')


In [13]:
# POISTA!
pd.set_option('display.max_row', None) 
pd.set_option('display.max_columns', None) 

In [14]:
testing_df.head()

,Kaupunginosa,Huoneisto,Talot.,m2,Rv,Krs,Hissi,Kunto,Asunnon tyyppi,Pituusaste,Leveysaste
0,Kaleva,"2h, k, rt, kph,...",kt,56.0,1956,3/4,on,tyyd.,Kaksi huonetta,23.805462,61.496083
1,Keskusta,"2h,kk,s",kt,56.0,1908,NaN,on,hyvä,Kaksi huonetta,24.064453,61.468052
2,Hämeenpuisto,"4-5 h,avok,2xkp...",kt,184.0,1928,3/6,on,hyvä,Neljä huonetta tai enemmän,23.751403,61.492285
3,Viinikka,"3h, k, kph, p",kt,65.0,1956,2/2,ei,hyvä,Kolme huonetta,23.786301,61.489502
4,Leinola,4 h + k + s + w...,ok,112.0,1978,NaN,ei,hyvä,Neljä huonetta tai enemmän,23.914912,61.490368


## Data Cleaning (optional)

In [15]:
import re
from sklearn.preprocessing import LabelEncoder

training_df['Rv'] = training_df['Rv'].astype(int)
testing_df['Rv'] = testing_df['Rv'].astype(int)

training_df['Kunto'] = training_df['Kunto'].fillna('Ei tietoa')
testing_df['Kunto'] = testing_df['Kunto'].fillna('Ei tietoa')

kunto_mappaus = {
    'Ei tietoa': 0,
    'huono': 1,
    'tyyd.': 2,
    'hyvä': 3,
}
training_df['Kunto'] = training_df['Kunto'].map(kunto_mappaus)
testing_df['Kunto'] = testing_df['Kunto'].map(kunto_mappaus)

le_kunto = LabelEncoder()
le_kunto.fit(list(training_df['Kunto'].values) + list(testing_df['Kunto'].values))
training_df['Kunto'] = le_kunto.transform(training_df['Kunto'])
testing_df['Kunto'] = le_kunto.transform(testing_df['Kunto'])

training_df['Hissi'] = training_df['Hissi'].astype('category')
training_df['Hissi'] = training_df['Hissi'].cat.codes

testing_df['Hissi'] = testing_df['Hissi'].astype('category')
testing_df['Hissi'] = testing_df['Hissi'].cat.codes

le_kaupunginosa = LabelEncoder()

training_df['Kaupunginosa'] = training_df['Kaupunginosa'].astype('category')   
testing_df['Kaupunginosa'] = testing_df['Kaupunginosa'].astype('category')

raja_arvo = 0.01
maarat = training_df['Kaupunginosa'].value_counts(normalize=True)
pienet_ryhmat = maarat[maarat < raja_arvo].index
training_df['Kaupunginosa'] = training_df['Kaupunginosa'].replace(pienet_ryhmat, 'Muu')

maarat = testing_df['Kaupunginosa'].value_counts(normalize=True)
pienet_ryhmat = maarat[maarat < raja_arvo].index
testing_df['Kaupunginosa'] = testing_df['Kaupunginosa'].replace(pienet_ryhmat, 'Muu')

training_df['Kaupunginosa'] = training_df['Kaupunginosa'].astype('category')
training_df['Kaupunginosa'] = training_df['Kaupunginosa'].cat.codes

testing_df['Kaupunginosa'] = testing_df['Kaupunginosa'].astype('category')
testing_df['Kaupunginosa'] = testing_df['Kaupunginosa'].cat.codes

le_kaupunginosa.fit(list(training_df['Kaupunginosa'].values) + list(testing_df['Kaupunginosa'].values))
training_df['Kaupunginosa'] = le_kaupunginosa.transform(training_df['Kaupunginosa'])
testing_df['Kaupunginosa'] = le_kaupunginosa.transform(testing_df['Kaupunginosa'])

le_asunnontyyppi = LabelEncoder()

tyyppimat = {
    'Yksiö' : 1,
    'Kaksi huonetta' : 2,
    'Kolme huonetta' : 3,
    'Neljä huonetta tai enemmän' : 4
}
training_df['Asunnon tyyppi'] = training_df['Asunnon tyyppi'].map(tyyppimat)
testing_df['Asunnon tyyppi'] = testing_df['Asunnon tyyppi'].map(tyyppimat)

le_asunnontyyppi.fit(list(training_df['Asunnon tyyppi'].values) + list(testing_df['Asunnon tyyppi'].values))
training_df['Asunnon tyyppi'] = le_asunnontyyppi.transform(training_df['Asunnon tyyppi'])
testing_df['Asunnon tyyppi'] = le_asunnontyyppi.transform(testing_df['Asunnon tyyppi'])


training_df['Huoneisto'] = training_df['Huoneisto'].str.lower()
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace(' ', '')
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('+', ',')
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('...', "")

testing_df['Huoneisto'] = testing_df['Huoneisto'].str.lower()
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace(' ', '')
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('+', ',')
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('...', "")

training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('/', ',')
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('[0-9]+h', '', regex=True)

testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('/', ',')
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('[0-9]+h', '', regex=True)

training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('^,', '',regex=True)
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('^[-0-9]+', '',regex=True)
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace(',$', '',regex=True)
training_df['Huoneisto'] = training_df['Huoneisto'].str.replace('^,', '',regex=True)

testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('^,', '',regex=True)
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('^[-0-9]+', '',regex=True)
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace(',$', '',regex=True)
testing_df['Huoneisto'] = testing_df['Huoneisto'].str.replace('^,', '',regex=True)

huoneisto_split = training_df['Huoneisto'].str.split(',')
huoneisto_split = huoneisto_split.apply(lambda lst: ["parveke" if  re.search('^p$|^parv$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["wc" if  re.search('^w$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["sauna" if  re.search('^s$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["2wc" if  re.search('^erill.wc$|^2xwc$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["kph" if  re.search('^kh$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["alkovi" if  re.search('^alk$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["lasit" if  re.search('^l', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["avok" if  re.search('^avokeitti&#$', item) else item for item in lst])

exploded = huoneisto_split.explode()
exploded_lkm = exploded.value_counts()

minimi_lkm = 10
suodatetut_idx = exploded_lkm[exploded_lkm >= minimi_lkm].index
suodatettu_lista = huoneisto_split.apply(lambda lst: [item for item in lst if item in suodatetut_idx])
suodatettu_lista = suodatettu_lista.apply(lambda lst: ["määrittämätön"] if len(lst) == 0 else lst)

training_df['Huoneisto'] = suodatettu_lista.apply(lambda lst: ','.join(lst))

training_encoded = training_df['Huoneisto'].str.get_dummies(sep=',')


huoneisto_split = testing_df['Huoneisto'].str.split(',')
huoneisto_split = huoneisto_split.apply(lambda lst: ["parveke" if  re.search('^p$|^parv$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["wc" if  re.search('^w$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["sauna" if  re.search('^s$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["2wc" if  re.search('^erill.wc$|^2xwc$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["kph" if  re.search('^kh$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["alkovi" if  re.search('^alk$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["lasit" if  re.search('^l', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["avok" if  re.search('^avokeitti&#$', item) else item for item in lst])

exploded = huoneisto_split.explode()
exploded_lkm = exploded.value_counts()

minimi_lkm = 10
suodatetut_idx = exploded_lkm[exploded_lkm >= minimi_lkm].index
suodatettu_lista = huoneisto_split.apply(lambda lst: [item for item in lst if item in suodatetut_idx])
suodatettu_lista = suodatettu_lista.apply(lambda lst: ["määrittämätön"] if len(lst) == 0 else lst)

testing_df['Huoneisto'] = suodatettu_lista.apply(lambda lst: ','.join(lst))

testing_encoded = testing_df['Huoneisto'].str.get_dummies(sep=',')

#  Varmista, että molemmat data frame -rakenteet sisältävät samat sarakkeet
missing_cols = set(training_encoded.columns) - set(testing_encoded.columns)
for c in missing_cols:
    testing_encoded[c] = 0

# Varmista myös, että testidata sisältää kaikki koulutusdatan sarakkeet
missing_cols = set(testing_encoded.columns) - set(training_encoded.columns)
for c in missing_cols:
    training_encoded[c] = 0

# Järjestä sarakkeet varmistaaksesi saman järjestyksen
training_encoded = training_encoded.reindex(sorted(training_encoded.columns), axis=1)
testing_encoded = testing_encoded.reindex(sorted(testing_encoded.columns), axis=1)

# Yhdistä alkuperäiseen DataFrameen
training_df = pd.concat([training_df.drop(['Huoneisto'], axis=1), training_encoded], axis=1)
testing_df = pd.concat([testing_df.drop(['Huoneisto'], axis=1), testing_encoded], axis=1)

le_talot = LabelEncoder()


training_df["Talot."] = training_df["Talot."].astype('category')
testing_df["Talot."] = testing_df["Talot."].astype('category')

le_talot.fit(list(training_df['Talot.'].values) + list(testing_df['Talot.'].values))
training_df['Talot.'] = le_talot.transform(training_df['Talot.'])
testing_df['Talot.'] = le_talot.transform(testing_df['Talot.'])


training_df["Krs"] = training_df["Krs"].fillna("0/0")
testing_df["Krs"] = testing_df["Krs"].fillna("0/0")

training_df["Krs"] = training_df["Krs"].str.replace('^-', '', regex=True)
testing_df["Krs"] = testing_df["Krs"].str.replace('^-', '', regex=True)

kerros_split = training_df["Krs"].str.split('/', expand=True)  
training_df["kerros"] = kerros_split[0].astype(int)
training_df["max_kerros"] = kerros_split[1].astype(int)    

kerros_split = testing_df["Krs"].str.split('/', expand=True)
testing_df["kerros"] = kerros_split[0].astype(int)
testing_df["max_kerros"] = kerros_split[1].astype(int)

training_df.drop(['Krs'], axis=1, inplace=True)
testing_df.drop(['Krs'], axis=1, inplace=True)


























## Feature Engineering (optional)

In [16]:
# Feature engineering/extraction/discovery to extract features new from the raw data.

## Data Modeling

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(training_df.drop('Hinta', axis=1), training_df['Hinta'])

pred = reg.predict(training_df.drop('Hinta', axis=1))

print(f"Mean squared error: {mean_squared_error(training_df['Hinta'], pred)}")
print(f'Mean absolute error: {mean_absolute_error(training_df["Hinta"], pred)}')    
print(f'R2 score: {r2_score(training_df["Hinta"], pred)}')

Mean squared error: 516.697515
Mean absolute error: 12.586
R2 score: 0.9739523219558454


In [18]:
# Make sure to store your predictions in the 'Hinta' attribute of the testing dataframe.

pred_testing = reg.predict(testing_df)

testing_df['Hinta'] = pred_testing


## Store the results

In [19]:
# !MAKE SURE TO NOT CHANGE THE CODE WITHIN THIS CELL!. 
testing_df.to_pickle("out.pkl")